In [1]:
# Import dependencies #

import altair as alt
import pandas as pd
import os
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from itertools import product
import seaborn as sns
from sklearn.feature_selection import mutual_info_classif
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score



In [2]:
### import data ###
# empty list to store data in 
storm_data = []

# path to folder
recent_storm_data = '..\Tornado Data\Past_5yr_Storm_data'

# recent files
for filename in os.listdir(recent_storm_data): # loop through files in folder
    filepath = os.path.join(recent_storm_data, filename)
    data = pd.read_csv(filepath, low_memory=False) # read in as CSV
    storm_data.append(data) # add data to the list


# combine the data frames
all_storm_data = pd.concat(storm_data, ignore_index=True)

# sample the data
all_storm_data.head()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,201905,9,1554,201905,9,1830,137295,824116,TEXAS,48,...,7.0,NNE,SAN GERONIMO,29.7898,-98.6406,29.7158,-98.7744,Thunderstorms developed along a cold front as ...,Thunderstorms produced heavy rain that led to ...,CSV
1,201908,1,0,201908,7,1400,141502,849617,SOUTH DAKOTA,46,...,3.0,W,BRUCE,44.5400,-96.9600,44.4300,-96.9400,Minor flooding slowly dwindled during early Au...,"A continuation of flooding from July, the Big ...",CSV
2,201909,25,1823,201909,25,1825,141998,852808,ARIZONA,4,...,24.0,S,OCOTILLO,32.8700,-111.8800,32.8788,-111.8750,Scattered thunderstorms developed over the cen...,Scattered thunderstorms developed across the c...,CSV
3,201902,19,2226,201902,19,2350,134941,808922,ARKANSAS,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Rain was heavy at times on the 19th, and there...",One-quarter inch of freezing rain was measured...,CSV
4,201902,19,2255,201902,19,2355,134941,808923,ARKANSAS,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Rain was heavy at times on the 19th, and there...",One-quarter inch of freezing rain was measured...,CSV


In [ ]:
all_storm_data.columns

In [ ]:
# filter for the rows with tornado data
tornado_df = all_storm_data[all_storm_data['EVENT_TYPE'] == 'Tornado']
tornado_df.count()

In [ ]:
# create new data frame without unneccesary columns
columns_to_keep = ['YEAR', 'MONTH_NAME', 'BEGIN_DAY', 'STATE', 'BEGIN_DATE_TIME',
                    'END_DATE_TIME', 'INJURIES_DIRECT', 'DEATHS_DIRECT', 'DAMAGE_PROPERTY',  'DAMAGE_CROPS', 'TOR_F_SCALE',
                    'TOR_LENGTH', 'TOR_WIDTH', 'BEGIN_RANGE', 'BEGIN_LAT', 'BEGIN_LON']

tornado_df = tornado_df[columns_to_keep]

tornado_df.columns

In [ ]:
# view data types to see if any need to be changed
tornado_df.dtypes

In [ ]:
len(tornado_df)

### Preprocessing ###

In [ ]:
tornado_df['BEGIN_DATE_TIME'] = tornado_df['BEGIN_DATE_TIME'].astype('datetime64[ns]')
tornado_df['END_DATE_TIME'] = tornado_df['END_DATE_TIME'].astype('datetime64[ns]')

In [ ]:
tornado_df.dtypes

In [ ]:
# check NA count
tornado_df.isna().sum()

In [ ]:
# only 9 entries null for lat and lon, so they can be dropped
tornado_df = tornado_df.dropna(subset = ['BEGIN_LAT', 'BEGIN_LON'])

# only null values left are in the damage columns, they can be filled with 0 because possible that they simply did no damage
tornado_df = tornado_df.fillna(0)

### EDA ###

In [ ]:
### calculate and identify total occurrences for each state or broader regions ###
### proportions for sampling purposes ###

### new column for broader region ###
REGIONS = {
    "NORTHEAST": ["CONNECTICUT", "DELAWARE", "MAINE", "MARYLAND", "MASSACHUSETTS", "NEW HAMPSHIRE", "NEW JERSEY", "NEW YORK", "PENNSYLVANIA", "RHODE ISLAND", "VERMONT"],
    "UPPER MIDWEST": ["IOWA", "MICHIGAN", "MINNESOTA", "WISCONSIN"],
    "OHIO VALLEY": ["ILLINOIS", "INDIANA", "KENTUCKY", "MISSOURI", "OHIO", "TENNESSEE", "WEST VIRGINIA"],
    "SOUTHEAST": ["ALABAMA", "FLORIDA", "GEORGIA", "NORTH CAROLINA", "SOUTH CAROLINA", "VIRGINIA"],
    "NORTHERN ROCKIES AND PLAINS": ["MONTANA", "NEBRASKA", "NORTH DAKOTA", "SOUTH DAKOTA", "WYOMING"],
    "SOUTH": ["ARKANSAS", "KANSAS", "LOUISIANA", "MISSISSIPPI", "OKLAHOMA", "TEXAS"],
    "SOUTHWEST": ["ARIZONA", "COLORADO", "NEW MEXICO", "UTAH"],
    "NORTHWEST": ["IDAHO", "OREGON", "WASHINGTON"],
    "WEST": ["CALIFORNIA", "NEVADA"]
}

state_to_region = {state: region for region, states in REGIONS.items() for state in states}

tornado_df['Region'] = tornado_df['STATE'].map(state_to_region)

tornado_df.head()

In [ ]:
### Potential 'tornado clusters' so only use first event for that day/time ###
tornado_df = tornado_df.groupby(['YEAR', 'MONTH_NAME', 'BEGIN_DAY', 'STATE'], as_index = False).first()

tornado_df.head()

Climate Regions as dictated by NCEI (https://www.ncei.noaa.gov/access/monitoring/reference-maps/us-climate-regions)

In [ ]:
### Regional distribution ###
region_counts = tornado_df['Region'].value_counts()

fig = go.Figure(data=[go.Bar(
    x=region_counts.index,
    y=region_counts.values,  
    marker_color='skyblue'   
)])

# Update layout with title and axis labels
fig.update_layout(
    title={
        'text': "Number of Events Per Region",
        'x': 0.5,         
        'xanchor': 'center'
    },
    xaxis_title='Region',
    yaxis_title='Number of Events',
    xaxis_tickangle=-45 
)

fig.show()

In [ ]:
### View the number of occurrences per month to show most likely months for activity ###

# make month name categorical so to sort in correct order
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
tornado_df['MONTH_NAME'] = pd.Categorical(tornado_df['MONTH_NAME'], categories=month_order, ordered=True)

monthly_occurrences = tornado_df['MONTH_NAME'].value_counts()
sorted_value_counts = monthly_occurrences.sort_index()

fig = go.Figure(data = [go.Line(
    x = sorted_value_counts.index,
    y = sorted_value_counts.values,
    mode = 'lines+markers',
    line = dict(color = 'thistle'), name = 'RGBA Color'
)]    
)

fig.update_layout(
    title={
        'text': "Monthly Occurrences",
        'x': 0.5,         
        'xanchor': 'center'
    },
    xaxis_title = 'Month',
    yaxis_title = 'Count'

)

fig.show()

In [ ]:
### Pie charts to show percentages for how frequent occurrences are for each month and each region seperately ###

regional_pie = px.pie(
    region_counts,
    values = region_counts.values,
    names = region_counts.index,
    color_discrete_sequence=px.colors.sequential.Greens[::-1]
)

regional_pie.update_traces(
    hovertemplate='%{label}: %{value}<extra></extra>',
    textposition = 'inside'
)

regional_pie.update_layout(
    uniformtext_minsize=12,
    uniformtext_mode='hide',
    width=750, 
    height=500, 
    margin=dict(
        l=75,  
        r=100,  
        t=100,  
        b=100    
    ),
    title={
        'text': "Events Per Region",
        'x': 0.5,           
        'xanchor': 'center'
        }
)

regional_pie.show()

#########

monthly_pie = px.pie(
    monthly_occurrences,
    values = monthly_occurrences.values,
    names = monthly_occurrences.index,
    color_discrete_sequence=px.colors.sequential.Blues[::-1]
)

monthly_pie.update_traces(
    hovertemplate='%{label}: %{value}<extra></extra>',
    textposition= 'inside'
)


monthly_pie.update_layout(
    uniformtext_minsize=12,
    uniformtext_mode='hide',
    width=750, 
    height=500, 
    margin=dict(
        l=75,  
        r=255,  
        t=100,  
        b=100    
    ),
    title={
        'text': "Events Per Month",
        'x': 0.5,           
        'xanchor': 'center'
        }
)

monthly_pie.show()

The intent was to create a synthetic dataset that contained every day from January 1, 2019 - December 31, 2023 that was comprised of every day other than the days of the tornado events so random samples of non-tornado days could be generated and selected because the original dataset contained only tornado event details. This dataframe of non-tornado days would be passed into a a seperate python script used to fetch weather data for those days.

In [ ]:
### Create non_tornado_df ###

# Column for each state #
states = [
    'ALABAMA', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA', 'COLORADO', 'CONNECTICUT', 'DELAWARE', 'FLORIDA', 'GEORGIA',
    'HAWAII', 'IDAHO', 'ILLINOIS', 'INDIANA', 'IOWA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND',
    'MASSACHUSETTS', 'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI', 'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEVADA', 'NEW HAMPSHIRE', 'NEW JERSEY',
    'NEW MEXICO', 'NEW YORK', 'NORTH CAROLINA', 'NORTH DAKOTA', 'OHIO', 'OKLAHOMA', 'OREGON', 'PENNSYLVANIA', 'RHODE ISLAND', 'SOUTH CAROLINA',
    'SOUTH DAKOTA', 'TENNESSEE', 'TEXAS', 'UTAH', 'VERMONT', 'VIRGINIA', 'WASHINGTON', 'WEST VIRGINIA', 'WISCONSIN', 'WYOMING'
]

# Every combination of date and state #
date_range = pd.date_range(start='2019-01-01', end='2023-12-31')
combinations = list(product(states, date_range))

df = pd.DataFrame(combinations, columns=['state', 'date'])
df = df.sort_values(['state', 'date']).reset_index(drop=True)

df.head()
len(df)

In [ ]:
# Map the region to the states for the non_tornado_df #

df['Region'] = df['state'].map(state_to_region)

In [ ]:
# Drop all day rows that exist in the tornado_df so that we know there is no overlap #

tornado_df['BEGIN_DATE'] = pd.to_datetime(tornado_df['BEGIN_DATE_TIME']).dt.date
tornado_dates = tornado_df['BEGIN_DATE'].unique()
df_no_tornados = df[~df['date'].isin(tornado_dates)]
df_no_tornados = df_no_tornados.reset_index(drop=True)

len(df_no_tornados)

In [ ]:
# Add month name column #

df_no_tornados['MONTH_NAME'] = df_no_tornados['date'].dt.strftime('%B')
df_no_tornados.head()

In [ ]:
# Create categorical data type for month name #
df_no_tornados['MONTH_NAME'] = pd.Categorical(df_no_tornados['MONTH_NAME'], categories=month_order, ordered=True)

In [ ]:
# Matrix function for simplicity of observing proportion distribution #

def create_matrix(df):
    matrix = pd.pivot_table(
        df,
        index='Region',        
        columns='MONTH_NAME',   
        aggfunc='size',       
        fill_value=0         
    )

    return matrix
region_matrix = create_matrix(tornado_df)


In [ ]:
# Calculate proportions matrix #

total_tornados = region_matrix.sum().sum()
proportions_matrix = region_matrix / total_tornados

proportion_series = proportions_matrix.stack()
proportion_series.index.names = ['Region', 'MONTH_NAME']

# Set batch parameters
batch_size = 1250
tornado_ratio = 0.3
tornado_size = int(batch_size * tornado_ratio)
non_tornado_size = batch_size - tornado_size

# Sample tornado days
tornado_samples = (proportion_series * tornado_size).round().astype(int)
tornado_samples = tornado_samples.where(tornado_samples > 0, 1 * (proportion_series > 0))

sampled_tornado_data = []
for (region, month), num_samples in tornado_samples.items():
    if num_samples > 0:
        subset = tornado_df[(tornado_df['Region'] == region) & (tornado_df['MONTH_NAME'] == month)]
        if len(subset) < num_samples:
            sampled_subset = resample(subset, n_samples=num_samples, replace=True)
        else:
            sampled_subset = subset.sample(n=num_samples, replace=False)
        sampled_tornado_data.append(sampled_subset)

# Sample non-tornado days
sampled_non_tornado_data = []
for region in region_matrix.index:
    for month in month_order:
        num_samples = max(1, int(non_tornado_size * proportion_series.get((region, month), 0)))
        subset = df_no_tornados[(df_no_tornados['Region'] == region) & (df_no_tornados['MONTH_NAME'] == month)]
        if len(subset) < num_samples:
            sampled_subset = resample(subset, n_samples=num_samples, replace=True)
        else:
            sampled_subset = subset.sample(n=num_samples, replace=False)
        sampled_non_tornado_data.append(sampled_subset)

# Combine tornado and non-tornado samples
sampled_tornado_df = pd.concat(sampled_tornado_data, ignore_index=True)
sampled_non_tornado_df = pd.concat(sampled_non_tornado_data, ignore_index=True)
len(sampled_non_tornado_df)


In [ ]:
### Create matrices and heat maps ###

def create_matrix(df):
    matrix = pd.pivot_table(
        df,
        index='Region',        
        columns='MONTH_NAME',     
        aggfunc='size',       
        fill_value=0          
    )

    return matrix

region_matrix = create_matrix(tornado_df)
region_matrix

In [ ]:
tornado_sample_matrix = create_matrix(sampled_tornado_df)
tornado_sample_matrix

In [ ]:
non_tornado_sample_matrix = create_matrix(sampled_non_tornado_df)
non_tornado_sample_matrix

In [ ]:
### Heatmap for clearer visual representation of distributions being the same ###

fig1 = px.imshow(
    region_matrix,
    labels={'x': 'State', 'y': 'Month', 'color': 'Number of Entries'},
    title='Tornado Events by Month and State'
)

fig1.update_layout(
    title={
        'text': "Events Per Month",
        'x': 0.45,
        'xanchor': 'center'
        }
) 

fig2 = px.imshow(
    tornado_sample_matrix,
    labels={'x': 'State', 'y': 'Month', 'color': 'Number of Entries'},
    title='Tornado Events by Month and State'
)

fig2.update_layout(
    title={
        'text': "Events Per Month",
        'x': 0.45,           
        'xanchor': 'center'
        }
)

fig3 = px.imshow(
    non_tornado_sample_matrix,
    labels={'x': 'State', 'y': 'Month', 'color': 'Number of Entries'},
    title='Tornado Events by Month and State'
)

fig3.update_layout(
    title={
        'text': "Events Per Month",
        'x': 0.45,           
        'xanchor': 'center'
        }
)

# Show the heatmap
fig1.show()
# Show the heatmap
fig2.show()

fig3.show()

In [ ]:
# Save to CSV for fetching data with api #

sampled_tornado_df.to_csv('API Data Request/tornado_sample.csv', index = False, header = True)
sampled_non_tornado_df.to_csv('API Data Request/non_tornado_sample.csv', index = False, header = True)

In [ ]:
# Import weather data that was fetched from external script #

tornado_weather_data = pd.read_csv('../Weather_API_DATA/tornado_weather_data.csv')
non_tornado_weather_data = pd.read_csv('../Weather_API_Data/non_tornado_weather_data.csv')

In [ ]:
tornado_weather_data.head()

In [ ]:
### Drop unnecessary columns ###
columns_to_drop = ['datetimeEpoch', 'sunrise', 'sunriseEpoch', 'sunset', 'sunsetEpoch', 'moonphase',
                    'description', 'stations', 'source', 'feelslikemin',
                   'feelslike', 'feelslikemax', 'snow', 'snowdepth',
                   'temp', 'severerisk', 'preciptype', 'precipcover', 'precipprob', 'Unnamed: 0'] ### unnamed 0: error from saving csv after API call

tornado_weather_data = tornado_weather_data.drop(columns = columns_to_drop)
non_tornado_weather_data = non_tornado_weather_data.drop(columns = columns_to_drop)
tornado_weather_data.head()

In [ ]:
# Map the regions back from the sample dataframe #
tornado_weather_data['Region'] = sampled_tornado_df['Region']
non_tornado_weather_data['Region'] = sampled_non_tornado_df['Region']

# add the tornado column
tornado_weather_data['Tornado'] = 1
non_tornado_weather_data['Tornado'] = 0

# calculate temp difference because this may indicate unstable environment rather than the individual temperatures of the day as they can differ throughout the seasons #
tornado_weather_data['Temp_Difference'] = tornado_weather_data['tempmax'] - tornado_weather_data['tempmin']
non_tornado_weather_data['Temp_Difference'] = non_tornado_weather_data['tempmax'] - non_tornado_weather_data['tempmin']
tornado_weather_data = tornado_weather_data.drop(columns=['tempmin', 'tempmax'])
non_tornado_weather_data = non_tornado_weather_data.drop(columns=['tempmin', 'tempmax'])

In [ ]:
# check NA #
tornado_weather_data.isna().sum()

In [ ]:
### Check which row is NA ###
rows_with_na = tornado_weather_data[tornado_weather_data['Region'].isna()]
rows_with_na

In [ ]:
tornado_weather_data['Region'].value_counts()

In [ ]:
sampled_tornado_df['Region'].value_counts()

In [ ]:
### Value counts were the same so dropping the one NA value for region ###

tornado_weather_data = tornado_weather_data.dropna(subset = ['Region'])
tornado_weather_data.isna().sum()

In [ ]:
### Concat both dataframes together ###
combined_weather_data = pd.concat([non_tornado_weather_data, tornado_weather_data], axis = 0)

In [ ]:
### Sort by date for simplicity ###
weather_by_date = combined_weather_data.sort_values(by = 'datetime')
weather_by_date.head()

In [ ]:
weather_by_date.isna().sum()

In [ ]:
### NA could represent no wind, fill with 0 ###

weather_by_date = weather_by_date.fillna(0)

In [ ]:
### Check datatypes ###
weather_by_date.dtypes

In [ ]:
weather_by_date.head()

In [ ]:
### Calculate correlation matrix ###

numeric_columns = ['dew', 'humidity','precip', 'windgust','windspeed', 'winddir', 'pressure', 'cloudcover','visibility', 'solarradiation',
'solarenergy', 'uvindex', 'Temp_Difference']

corr_matrix = weather_by_date[numeric_columns].corr()

corr_matrix

In [ ]:
### Plot heatmap for visual ###

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Heatmap of Numeric Features')
plt.show()

In [ ]:
### Feature importance ###

# Drop target and datetime
X = weather_by_date.drop(['Tornado', 'datetime'], axis=1)
y = weather_by_date['Tornado']

# Encode categoricals
X = pd.get_dummies(X, columns=['icon', 'Region', 'conditions'])

# Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X, y)

importance = rf.feature_importances_
feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': importance})
feature_importance = feature_importance.sort_values('Importance', ascending=False).reset_index(drop=True)

plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance.head(15))
plt.title('Top 15 Features by Random Forest Feature Importance')
plt.show()

In [ ]:
# Drop bottom columns
feature_selection = weather_by_date.drop(columns = ['solarradiation', 'Temp_Difference', 'solarenergy','visibility','conditions', 'uvindex'])

In [ ]:
# Assign X and Y
X = feature_selection.drop(['Tornado', 'datetime'], axis=1)
y = feature_selection['Tornado']

# Encode categorical variables
X = pd.get_dummies(X, columns=['Region', 'icon'])

# # Scale numeric columns
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Split train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y)
scaler = StandardScaler()

# Balance using SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
X_train_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)

# Adjust class weights and create the model
class_weight = {0: 1, 1: 5} 
rf_model = RandomForestClassifier(
    n_estimators=500, 
    max_depth=None,
    min_samples_leaf=1,
    max_features='sqrt',
    random_state=42,
    class_weight=class_weight
)

# Fit to the training data
rf_model.fit(X_train_scaled, y_train_resampled)

# Make predictions
y_pred = rf_model.predict(X_test_scaled)
y_pred_proba = rf_model.predict_proba(X_test_scaled)[:, 1]
y_pred_custom = (y_pred_proba > 0.5).astype(int)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred_custom))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nROC AUC Score:")
print(roc_auc_score(y_test, y_pred_proba))


In [ ]:
cv_scores = cross_val_score(rf_model, X, y, cv=5)
print("\nCross-validation scores:", cv_scores)
print("Mean CV score:", cv_scores.mean())

In [ ]:
# Identify new feature importance using the model
feature_importance = pd.DataFrame({'feature': X.columns, 'importance': rf_model.feature_importances_})
feature_importance = feature_importance.sort_values('importance', ascending=False)
print("\nTop 10 important features:")
print(feature_importance.head(10))

In [ ]:
# Hyperparameter tuning
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'class_weight': [{0:1, 1:2}, {0:1, 1:3}, {0:1, 1:5}]
}

rf_random = RandomizedSearchCV(RandomForestClassifier(random_state=42), 
                               param_distributions=param_dist, 
                               n_iter=100, cv=5, verbose=2, 
                               random_state=42, n_jobs=-1,
                               scoring='recall')  # Focus on recall
rf_random.fit(X_train_scaled, y_train_resampled)

print("Best parameters:", rf_random.best_params_)

In [ ]:
# Reevaluate the model using the best parameters from the RandomSearchCV
y_pred_best = rf_random.predict(X_test_scaled)
y_pred_proba_best = rf_random.predict_proba(X_test_scaled)[:, 1]

print("\nBest Model Classification Report:")
print(classification_report(y_test, y_pred_best))

print("\nBest Model ROC AUC Score:")
print(roc_auc_score(y_test, y_pred_proba_best))

print("\nBest Model Confusion Matrix")
print(confusion_matrix(y_test, y_pred_best))